### Réapprentissage automatique

In [6]:
import sys
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider 
import csv
import pandas as pd
import numpy as np

In [7]:
# Cassandra metadata
dev_cassandra_host= 'dtl-cassandra01-d01'
dev_cassandra_port= 9200
dev_username= 'loaddata'
dev_password= 'DecujRiQuigByaibdednofVerr6Odij2'

# Cassandra Keyspaces
parcours_keyspace = 'parcours'
referentiel_keyspace = 'ref_data'

# Cassandra tables
insee_table = 'ref_insee'
prospect_table = 'referentiel_prospects'
valo_retro_one_month_table = 'retro_valo_client_un_mois'

# Elasticsearch hostname
dev_es_host= 'dtl-esmaster01-d01'

# Elasticsearch port
dev_es_port= 9200

# Path to pickle 
path_to_one_month_pickle = "/home/slazrak/projet-encours/dtl-encours/MODEL/ENCOURS_1MOIS/"

# One month model target name
one_month_target_name = 'ENCOURS_1MOIS'

In [10]:
class CassandraClient:

    def __init__(self, keyspace):
        self.auth_provider = None
        self.cluster = None
        self.session = None
        self.keyspace = keyspace
        self.create_session()
        self.prepare_statements()

    def create_session(self):
        self.auth_provider = PlainTextAuthProvider(username=dev_username, password=dev_password)
        self.cluster = Cluster(contact_points = [dev_cassandra_host], protocol_version=3, auth_provider=self.auth_provider)
        self.session = self.cluster.connect(parcours_keyspace)

    def get_session(self):
        return self.session

    def prepare_statements(self):
        ### prospects referential
        self.statement_get_prospects = self.session.prepare('select * from ' + prospect_table + ' WHERE csp>-1 ALLOW FILTERING')
        ### Get real "encours" from Retro 1 month  
        self.statement_get_valo_retro_one_month = self.session.prepare('select * from ' + valo_retro_one_month_table + ' WHERE encours>-1 ALLOW FILTERING')
        ### get Insee for enrichment
        self.statement_get_insee = self.session.prepare('select * from ' + referentiel_keyspace + '.' + insee_table)
          

    def get_df_from_query(self,prepared_query):
        resultset = self.session.execute(prepared_query)
        resultset[0]
        df = pd.DataFrame()
        for num_col in range(len(resultset.column_names)):
            result_col = []
            for row in resultset:
                result_col.append(row[num_col])
            df[resultset.column_names[num_col]] = result_col
        return df



In [11]:
#cc = CassandraClient(parcours_keyspace)

### Get prospects' referential 
cc = CassandraClient(parcours_keyspace)
prospects_ref = cc.get_df_from_query(cc.statement_get_prospects)
#prospects_ref = pd.read_csv('/home/ubuntu/Bureau/ENCOURS_1MOIS_v2/ref_prospect.csv', sep=',')
prospects_ref.head()

contactid  annee_mois  campagne                            campaign  \
0  8091134532      201707      2071                  Libellé non trouvé   
1  7960001875      201706       NaN                                None   
2  7923207940      201706       NaN                                None   
3  7939163329      201706      1202  REC_Onlin_Parte_Panor_Bours_banque   
4  8090109164      201707       306                REC_Online_Yahoo SEM   

  civilite code_offre code_postal code_postal_naissance  \
0      MLE       None       90000                 71200   
1     None       None        None                 59000   
2     None       None        None                 94400   
3      MLE       None       97421                 97421   
4      MME       None       78125                 66000   

  country_connexion_name csl_flag              ...               namesponsor  \
0                 France     None              ...                      None   
1                   None     None              ...                    DERIAU   
2                   None     None              ...                      None   
3                Réunion     None              ...                      None   
4                 France     None              ...                      None   

  nature_cb patrimoine  pays  pays_naissance prevoyance_flag  \
0      None          0   FRA             FRA            None   
1      None          0  None            None            None   
2      None          1  None            None            None   
3      None          1  None            None            None   
4      None          2   FRA             FRA            None   

   regime_matrimonial revenus_annuels  \
0                   6               0   
1                   0               0   
2                   2               2   
3                   0               1   
4                   2               4   

                                             service  \
0  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   
1  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   
2  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   
3  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   
4  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   

                      timestamp  
0  2017-07-03T22:10:48.393+0200  
1  2017-06-12T18:13:55.754+0200  
2  2017-06-06T14:58:51.909+0200  
3  2017-06-08T15:42:35.847+0200  
4  2017-07-03T11:19:27.066+0200  

[5 rows x 29 columns]

In [12]:
### Get real 'encours' of customers from Retro 
retro = cc.get_df_from_query(cc.statement_get_valo_retro_one_month)
#retro = pd.read_csv('/home/ubuntu/Bureau/ENCOURS_1MOIS_v2/retro.csv', sep=',')
retro.head()

id_dim_personne  annee_mois  annee_mois_m_x  cible_encours_predit  \
0       7960293527      201707        20170725                     1   
1       6503940634      201707        20170725                     0   
2       7889215540      201707        20170725                     0   
3       8090683890      201706        20170629                     0   
4       7991769664      201707        20170725                     1   

   cible_reelle date_entree_relation delai_transfo   encours id_dim_temps  \
0             1           2017-06-12          None  13346.00   2017-06-12   
1             0           2016-09-12          None     10.00   2017-05-22   
2             0           2017-05-31          None    887.27   2017-05-31   
3             0           2015-07-03          None    298.50   2017-07-03   
4             0           2017-06-16          None    288.21   2017-06-16   

   identifiant_web score_prospect_encours  x_mois_anterieurs_cumules  
0         25277565                   None                          1  
1         25937634                   None                          1  
2         25987519                   None                          1  
3         88107541         0.336370825768                          1  
4         29127501                   None                          1

In [13]:
#Delete Unnecessary fields from retro
del retro['annee_mois']
del retro['annee_mois_m_x']
del retro['cible_encours_predit']
del retro['date_entree_relation']
del retro['delai_transfo']
del retro['id_dim_temps']
del retro['identifiant_web']
del retro['score_prospect_encours']
del retro['x_mois_anterieurs_cumules']
retro['cible'] = retro['cible_reelle']
del retro['cible_reelle']

In [14]:
retro.head()

id_dim_personne   encours  cible
0       7960293527  13346.00      1
1       6503940634     10.00      0
2       7889215540    887.27      0
3       8090683890    298.50      0
4       7991769664    288.21      0

In [23]:
prospects_ref.contactid = pd.to_numeric(prospects_ref.contactid)

In [25]:
### Merge tables
merge_1 = pd.merge(prospects_ref,retro, right_on='id_dim_personne', left_on='contactid')
merge_1.head()

contactid  annee_mois  campagne                  campaign civilite  \
0  7932573625      201706      1437  REC_Onlin_Parte_Bours_VP      MLE   
1  7939180599      201706       868     PAR_Onlin_Site _ECard       MR   
2  7904191160      201706       300         Intb_Banque_Autre       MR   
3  7955059788      201706       868     PAR_Onlin_Site _ECard       MR   
4  7990238161      201706       300         Intb_Banque_Autre       MR   

  code_offre code_postal code_postal_naissance country_connexion_name  \
0        NaN       17140                 29200                 France   
1        NaN       75005                 76000                 France   
2        NaN       74370                 74000                 France   
3        NaN       94200                 75015                 France   
4        NaN       33470                 91600                 France   

  csl_flag  ...   pays pays_naissance prevoyance_flag regime_matrimonial  \
0      NaN  ...   None           None             NaN                  0   
1      NaN  ...   None           None             NaN                  0   
2      NaN  ...   None           None             NaN                  4   
3      NaN  ...   None           None             NaN                  0   
4      NaN  ...   None           None             NaN                  2   

   revenus_annuels                                            service  \
0                1  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   
1                2  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   
2                1  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   
3                1  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   
4                1            monprofil.PROSPECT/CREER_FICHE_PROSPECT   

                      timestamp id_dim_personne  encours cible  
0  2017-06-07T18:44:52.043+0200      7932573625   450.00     0  
1  2017-06-10T14:31:42.001+0200      7939180599   748.08     0  
2  2017-06-02T12:01:00.383+0200      7904191160   362.54     0  
3  2017-06-12T01:07:46.129+0200      7955059788  2010.00     1  
4  2017-06-16T12:56:40.857+0200      7990238161   833.53     0  

[5 rows x 32 columns]

In [27]:
### Get INSEE for enrichment 
#cc2 = CassandraClient(referentiel_keyspace)
insee_ref = cc.get_df_from_query(cc.statement_get_insee)
#insee_ref = pd.read_csv('/home/ubuntu/Bureau/ENCOURS_1MOIS_v2/Referentiel1.csv', sep = ',')
#insee_ref['code_postal'] = insee_ref['Code_geographique']
#del insee_ref['Libelle_geographique'] 
#del insee_ref['Code_geographique'] 
insee_ref.head()

code_postal  salaire_net_horaire_moyen_en_2014_euro
0        27120                                    14.2
1        21220                                    13.1
2        57210                                    12.3
3        57190                                    12.2
4        47110                                    10.6

In [31]:
merge_1.code_postal = pd.to_numeric(merge_1.code_postal)

In [32]:
### Merge previous table with insee
merge_2 = pd.merge(merge_1,insee_ref, right_on= 'code_postal', left_on= 'code_postal')
merge_2.head()

contactid  annee_mois  campagne                  campaign civilite  \
0  7932573625      201706      1437  REC_Onlin_Parte_Bours_VP      MLE   
1  7939180599      201706       868     PAR_Onlin_Site _ECard       MR   
2  7904191160      201706       300         Intb_Banque_Autre       MR   
3  7904196332      201706       300         Intb_Banque_Autre      MLE   
4  7955059788      201706       868     PAR_Onlin_Site _ECard       MR   

  code_offre  code_postal code_postal_naissance country_connexion_name  \
0        NaN        17140                 29200                 France   
1        NaN        75005                 76000                 France   
2        NaN        74370                 74000                 France   
3        NaN        74370                 74100                 France   
4        NaN        94200                 75015                 France   

  csl_flag                  ...                    pays_naissance  \
0      NaN                  ...                              None   
1      NaN                  ...                              None   
2      NaN                  ...                              None   
3      NaN                  ...                              None   
4      NaN                  ...                              None   

  prevoyance_flag regime_matrimonial revenus_annuels  \
0             NaN                  0               1   
1             NaN                  0               2   
2             NaN                  4               1   
3             NaN                  4               0   
4             NaN                  0               1   

                                             service  \
0  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   
1  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   
2  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   
3  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   
4  monprofil.CONTACT_MANAGEMENT/amendContactInfor...   

                      timestamp  id_dim_personne  encours cible  \
0  2017-06-07T18:44:52.043+0200       7932573625   450.00     0   
1  2017-06-10T14:31:42.001+0200       7939180599   748.08     0   
2  2017-06-02T12:01:00.383+0200       7904191160   362.54     0   
3  2017-06-02T12:03:24.226+0200       7904196332    10.00     0   
4  2017-06-12T01:07:46.129+0200       7955059788  2010.00     1   

  salaire_net_horaire_moyen_en_2014_euro  
0                                   15.1  
1                                   22.2  
2                                   15.3  
3                                   15.3  
4                                   14.0  

[5 rows x 33 columns]

In [33]:
merge_2 = merge_2.set_index('contactid')

In [34]:
del merge_2['cible']

In [35]:
#Fix encours of good costumers (1200) 
merge_2['cible_seuil_1200'] = merge_2['encours'].map(lambda x: 0 if x < 1200.00 else 1).astype(int)
merge_2[['encours','cible_seuil_1200']].head()

encours  cible_seuil_1200
contactid                            
7932573625   450.00                 0
7939180599   748.08                 0
7904191160   362.54                 0
7904196332    10.00                 0
7955059788  2010.00                 1

In [45]:
retro.shape

(80, 3)

In [36]:
### Data management
def data_management(df):
    
    ### Get flag parrain from namesponsor
    df['Parrain'] = df['namesponsor'].map(lambda x: 0 if pd.isnull(x) else 1).astype(int)
    del df['firstnamesponsor'] 
    del df['namesponsor'] 
    
    ### Get domain from mail
    df['mail'] = df.mail.replace ([np.nan], '')
    df['domain'] = df['mail'].map(lambda x: x.split("@")[1] if x else None)
    df['domain'] = df.domain.str.lower()   

    ### Get Sex from Civility
    df['Sexe'] = df['civilite'].map(get_sex)
    
    ### Delete useless cols
    del df['civilite']
    del df['mail']
    del df['annee_mois']
    del df['mailing_accord']
    del df['id_dim_temps']
    del df['id_dim_personne']
    del df['encours']
    del df['timestamp']
    del df['service']
    del df['campaign']
    #del df['contactid']
    del df['pays']
    del df['debit_cb']
    del df['code_postal']
    del df['nature_cb']
    
    ### Impute empty rows  
    df['date_naissance'] = df['date_naissance'].replace('','01/01/1800')
    df = df.replace ([np.nan,None,''], -2)
    
    ### Get age from birthdate
    get_age(df)

    ### transform to numeric when possible
    df = df.apply(lambda x: pd.to_numeric(x, errors='ignore'))
    
    ### filter the most reccurent ones
    df.loc[df['country_connexion_name'].value_counts()[df['country_connexion_name']].values < 40, 'country_connexion_name'] = 'other'
    df.loc[df['domain'].value_counts()[df['domain']].values < 40, 'domain'] = 'other'
    df.loc[df['pays_naissance'].value_counts()[df['pays_naissance']].values < 40, 'pays_naissance'] = 'other'
     
    ### Get country connexion name
    df = process_country_connexion(df)
    
    ### Get pays de naissance
    df = process_pays_naissance(df)
    #df['pays_de_naissance'] = df['pays_naissance'].map(process_pays_naissance)
    #del df['pays_naissance']
    
    ### Process the domain
    df = process_domain (df)
    
    ### Drop NaN 
    #df = df.dropna()
    df = df.replace ([np.nan,None,''], -2)
    
    features_df = df.drop('cible_seuil_1200', axis=1)
    target = df['cible_seuil_1200']
    return features_df, target, df


def get_sex(x):
    if x == "0":
        return 1
    elif (x == "1"):
        return 0
    elif (x == "2"):
        return 0
    elif (x == "MR"):
        return 1
    elif (x == "MLE"):
        return 0
    elif (x == "MME"):
        return 0
    else :
        return -1

def get_age(X):
    from datetime import datetime
    now = datetime.now()
    X['AGE'] = X['date_naissance'].dropna()
    X['AGE'] = pd.to_datetime(X['AGE'],errors='coerce')
    X['AGE'] = X['AGE'].map(lambda x : now.year-x.year  if now.month-x.month>0 else now.year - x.year -1 )
    del X['date_naissance']

def process_country_connexion(df):
    """
    Process country_connexion
    """
    dict_ = {'France':1, 'United Kingdom':2, 'other': 3, 'Germany': 4, 'Switzerland': 5, 'Netherlands': 6, 'Europe': 7, 'United States': 8, 'Reunion': 9, -2: -2}
    df['country_connexion'] = df['country_connexion_name'].map(dict_)
    del df['country_connexion_name']
    return df
    
def process_pays_naissance(df):
    """
    Process pays naissance
    """
    dict_ = {'FRA':1, 'ITA':2, 'other': 3, 'DZA': 4, 'BEN': 5, 'ESP': 6, 'VNM': 7, 'DEU': 8, 'MAR': 9, 'CIV': 10, 'CMR': 11, 'GTO': 12,
            'SEN': 13,'BEL': 14, 'CHN': 15, 'ROU':16, 'BRA': 17, 'MDG': 18,
            'PRT': 19, 'GBR': 20, 'LBN': 21, 'TUR': 22, 'IND': 23}
    df['pays_de_naissance'] = df['pays_naissance'].map(dict_)
    del df['pays_naissance']
    return df
  
def process_domain(df):
    """
    Process the domain features
    """
    dict_ = {'yahoo.fr':0, 'hotmail.fr':1, 'hotmail.com': 2, 'gmail.com': 3, 'orange.fr': 4, 'outlook.com': 5, 'free.fr': 6, 'laposte.net': 7, 'other': 8, 'neuf.fr': 9, 'wanadoo.fr': 10, 'me.com': 11, 'ymail.com': 12, 'sfr.fr': 13, 'live.fr': 14, 'bbox.fr': 15, 'outlook.fr': 16, 'msn.com': 17, 'yahoo.com': 18, 'aol.com': 19, 'icloud.com': 20, 'cegetel.net': 21, 'club-internet.fr': 22}
    df['domain'] = df['domain'].map(dict_)
    return df

In [37]:
X, y, data= data_management(merge_2)

/opt/anaconda/lib/python2.7/site-packages/pandas/core/common.py:449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


In [38]:
print ('Types', data.dtypes)
print ('X.shape', X.shape)
print ('y.shape', y.shape)

('Types', campagne                                  float64
code_offre                                  int64
code_postal_naissance                       int64
csl_flag                                    int64
csp                                       float64
flag_banque_principale                    float64
flag_epargne                                int64
patrimoine                                float64
prevoyance_flag                             int64
regime_matrimonial                        float64
revenus_annuels                           float64
salaire_net_horaire_moyen_en_2014_euro    float64
cible_seuil_1200                            int64
Parrain                                     int64
domain                                      int64
Sexe                                        int64
AGE                                         int64
country_connexion                           int64
pays_de_naissance                         float64
dtype: object)
('X.shape', (79, 18))
('y

In [41]:
# Modèle : nearest_centroid
from sklearn.neighbors.nearest_centroid import NearestCentroid
classifier = NearestCentroid(metric='euclidean', shrink_threshold = 1.5)
classifier.fit(X,y)

NearestCentroid(metric='euclidean', shrink_threshold=1.5)

In [60]:
## save the model to disk
import cPickle
with open('Model_v2.pkl', 'wb') as f:
    cPickle.dump(classifier, f)